In [3]:
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

# Define the transformation matrix M (3x3 for 3D transformations)
M = np.array([[0.6, 0.3, 0.2],
              [-0.2, 1.4, 0.1],
              [0.1, -0.1, 1.1]])

# Function to generate sequences
def generate_sequence(point, matrix, steps=40):
    sequence = [point]
    for _ in range(steps - 1):
        point = matrix @ point
        sequence.append(point)
    return np.array(sequence)

# Create Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("Interactive 3D Transformation Model (Default X-Y View)"),
    dcc.Graph(
        id="3d-plot",
        config={"scrollZoom": True}  # Enables zooming and rotating in 3D
    ),
    html.Div([
        html.Label("Adjust Coordinates of Point A:"),
        dcc.Slider(id="slider-ax", min=-60, max=60, step=0.1, value=28.82,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
        dcc.Slider(id="slider-ay", min=-60, max=60, step=0.1, value=-0.86,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
        dcc.Slider(id="slider-az", min=-60, max=60, step=0.1, value=10.0,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
    ]),
    html.Div([
        html.Label("Adjust Coordinates of Point B:"),
        dcc.Slider(id="slider-bx", min=-60, max=60, step=0.1, value=19.71,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
        dcc.Slider(id="slider-by", min=-60, max=60, step=0.1, value=3.79,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
        dcc.Slider(id="slider-bz", min=-60, max=60, step=0.1, value=15.0,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
    ]),
    html.Div([
        html.Label("Adjust Coordinates of Point C:"),
        dcc.Slider(id="slider-cx", min=-60, max=60, step=0.1, value=40.96,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
        dcc.Slider(id="slider-cy", min=-60, max=60, step=0.1, value=6.62,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
        dcc.Slider(id="slider-cz", min=-60, max=60, step=0.1, value=20.0,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
    ]),
])

# Callback to update the 3D plot
@app.callback(
    Output("3d-plot", "figure"),
    [Input("slider-ax", "value"), Input("slider-ay", "value"), Input("slider-az", "value"),
     Input("slider-bx", "value"), Input("slider-by", "value"), Input("slider-bz", "value"),
     Input("slider-cx", "value"), Input("slider-cy", "value"), Input("slider-cz", "value")]
)
def update_plot(ax, ay, az, bx, by, bz, cx, cy, cz):
    # Points A, B, C
    A = np.array([ax, ay, az])
    B = np.array([bx, by, bz])
    C = np.array([cx, cy, cz])
    
    # Generate sequences
    sequence_A = generate_sequence(A, M)
    sequence_B = generate_sequence(B, M)
    sequence_C = generate_sequence(C, M)
    
    # Get eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(M)

    # Create plot
    fig = go.Figure()

    # Add sequences
    fig.add_trace(go.Scatter3d(
        x=sequence_A[:, 0], y=sequence_A[:, 1], z=sequence_A[:, 2],
        mode='lines+markers',
        name=f'Sequence A (A: [{A[0]:.2f}, {A[1]:.2f}, {A[2]:.2f}])',
        marker=dict(size=3, color='red')
    ))
    fig.add_trace(go.Scatter3d(
        x=sequence_B[:, 0], y=sequence_B[:, 1], z=sequence_B[:, 2],
        mode='lines+markers',
        name=f'Sequence B (B: [{B[0]:.2f}, {B[1]:.2f}, {B[2]:.2f}])',
        marker=dict(size=3, color='blue')
    ))
    fig.add_trace(go.Scatter3d(
        x=sequence_C[:, 0], y=sequence_C[:, 1], z=sequence_C[:, 2],
        mode='lines+markers',
        name=f'Sequence C (C: [{C[0]:.2f}, {C[1]:.2f}, {C[2]:.2f}])',
        marker=dict(size=3, color='green')
    ))

    # Add infinite eigenvectors
    for i in range(3):
        t = np.linspace(-100, 100, 2)  # Extend infinitely in both directions
        fig.add_trace(go.Scatter3d(
            x=t * eigenvectors[0, i],
            y=t * eigenvectors[1, i],
            z=t * eigenvectors[2, i],
            mode='lines',
            line=dict(color='purple', width=2, dash='dash'),
            name=f'Eigenvector {i+1}'
        ))

    # Add points
    fig.add_trace(go.Scatter3d(
        x=[A[0]], y=[A[1]], z=[A[2]],
        mode='markers+text',
        marker=dict(size=8, color='red'),
        text=['A'],
        name='Point A'
    ))
    fig.add_trace(go.Scatter3d(
        x=[B[0]], y=[B[1]], z=[B[2]],
        mode='markers+text',
        marker=dict(size=8, color='blue'),
        text=['B'],
        name='Point B'
    ))
    fig.add_trace(go.Scatter3d(
        x=[C[0]], y=[C[1]], z=[C[2]],
        mode='markers+text',
        marker=dict(size=8, color='green'),
        text=['C'],
        name='Point C'
    ))

    # Layout settings with default X-Y plane view
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X', range=[-60, 60]),
            yaxis=dict(title='Y', range=[-60, 60]),
            zaxis=dict(title='Z', range=[-60, 60]),
            camera=dict(
                eye=dict(x=0, y=0, z=2)  # Set the camera to look from above (X-Y plane)
            )
        ),
        title='Interactive 3D Transformation Model (Default X-Y View)',
        showlegend=True
    )

    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'